In [12]:
from langchain_community.document_loaders import PyPDFLoader

file_PDF = r'data\tinh_huong/output.pdf'
raw_documents = PyPDFLoader(file_PDF).load()

In [13]:
# Hiển thị nội dung của một số trang đầu tiên
for i, document in enumerate(raw_documents[:5]):  # Lấy 5 trang đầu
    print(f"Trang {i + 1}:")
    print(document.page_content)
    print("\n" + "-" * 40 + "\n")

Trang 1:
Situation:
Tôi xin hỏi, học sinh vừa tốt nghiệp THPT khi xét tuyển vào các trường cao đẳng thì có được tạm
miễn thực hiện nghĩa vụ quân sự không?
Answer:
Bộ Quốc phòng trả lời vấn đề này như sau:Theo quy định tại Điểm g Khoản 1 Điều 41Luật Nghĩa vụ
quân sựnăm 2015, những công dân sau đây được tạm hoãn gọi nhập ngũ:"g) Đang học tại cơ sở
giáo dục phổ thông; đang được đào tạo trình độ đại học hệ chính quy thuộc cơ sở giáo dục đại học,
trình độ cao đẳng hệ chính quy thuộc các cơ sở giáo dục nghề nghiệp trong một khóa đào tạo của
một trình độ đào tạo".Như vậy, công dân thuộc trường hợp nêu tại Điểm g Khoản 1 Điều 41 Luật
Nghĩa vụ quân sự năm 2015 được tạm hoãn từ ngày công dân đã làm xong thủ tục nhập học và
đang học tập tại trường; chỉ được tạm hoãn gọi nhập ngũ trong một khóa đào tạo tập trung đầu
tiên, còn các khóa học tiếp theo thì không được tạm hoãn.

----------------------------------------

Trang 2:
Situation:
Hiện  nay  khi  tra  cứu  trên  Hệ  thống  thông  tin  giải  qu

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Điều chỉnh kích thước đoạn văn cho phù hợp với nội dung dài
    chunk_overlap=200  # Đảm bảo các đoạn ngắn không bị mất ngữ cảnh
)
docs = text_splitter.split_documents(raw_documents)

In [5]:
docs[0]

Document(page_content='Situation:\nTôi xin hỏi, học sinh vừa tốt nghiệp THPT khi xét tuyển vào các trường cao đẳng thì có được tạm\nmiễn thực hiện nghĩa vụ quân sự không?\nAnswer:\nBộ Quốc phòng trả lời vấn đề này như sau:Theo quy định tại Điểm g Khoản 1 Điều 41Luật Nghĩa vụ\nquân sựnăm 2015, những công dân sau đây được tạm hoãn gọi nhập ngũ:"g) Đang học tại cơ sở\ngiáo dục phổ thông; đang được đào tạo trình độ đại học hệ chính quy thuộc cơ sở giáo dục đại học,\ntrình độ cao đẳng hệ chính quy thuộc các cơ sở giáo dục nghề nghiệp trong một khóa đào tạo của\nmột trình độ đào tạo".Như vậy, công dân thuộc trường hợp nêu tại Điểm g Khoản 1 Điều 41 Luật\nNghĩa vụ quân sự năm 2015 được tạm hoãn từ ngày công dân đã làm xong thủ tục nhập học và\nđang học tập tại trường; chỉ được tạm hoãn gọi nhập ngũ trong một khóa đào tạo tập trung đầu\ntiên, còn các khóa học tiếp theo thì không được tạm hoãn.', metadata={'source': 'data\\tinh_huong/output.pdf', 'page': 0})

In [6]:
from transformers import AutoModel, AutoTokenizer
import torch

# Tải model PhoBERT-large
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-large")
model = AutoModel.from_pretrained("vinai/phobert-large")

def phobert_embedding(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # Lấy embeddings từ token [CLS]

embeddings = [phobert_embedding(doc.page_content) for doc in docs]

d:\Download_Python\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


IndexError: index out of range in self

In [11]:
from transformers import AutoModel, AutoTokenizer
import torch
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Tải mô hình PhoBERT-large và tokenizer
model_name = "vinai/phobert-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()  # Đặt mô hình ở chế độ đánh giá

# Hàm tạo embeddings với batch processing
def phobert_embedding(texts, batch_size=8):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        
        # Chuyển các tensors sang GPU nếu có, nếu không thì giữ lại trên CPU
        input_ids = input_ids.to(model.device)
        attention_mask = attention_mask.to(model.device)
        
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # Lấy embeddings từ token [CLS]
        batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        all_embeddings.extend(batch_embeddings)
    return all_embeddings

# Tải dữ liệu từ file PDF
file_PDF = r'data\tinh_huong/output.pdf'
raw_documents = PyPDFLoader(file_PDF).load()

# Chia dữ liệu thành các đoạn văn bản
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
docs = text_splitter.split_documents(raw_documents)
texts = [doc.page_content for doc in docs]

# Tạo embeddings cho các văn bản
embeddings = phobert_embedding(texts, batch_size=8)

print("Embeddings đã được tạo thành công.")


RuntimeError: The expanded size of the tensor (281) must match the existing size (258) at non-singleton dimension 1.  Target sizes: [8, 281].  Tensor sizes: [1, 258]

In [ ]:
###